### Optimizer description: ZSCG
**Name:**  Zero Stochastic Conditional Gradient<br>
**Class:** zeroOptim.ClassicZSCG <br>
**Paper:** *Zeroth-order Nonconvex Stochastic Optimization: Handling Constraints, High-Dimensionality and Saddle-Points* (rishnakumar Balasubramanian†1 and Saeed Ghadimi‡2) <br>


**Description:** <br>
The Zero-order Stochastic Conditional Gradient Descent at each iteration *k* try to minimize *F(z)* with these 3 main steps:

    1. Estimate the gradient as follow:
$$G_{v}^{k} \equiv G_{v}(z_{k-1}, \xi_{k}, u_{k}) = \frac{1}{m_{k}} \sum_{j=1}^{m_{k}} \frac{F(z_{k-1} + vu_{k,j}, \xi_{k,j}) - (z_{k-1}, \xi_{k,j})}{v}u_{k,j}$$
    2. Solve this linear programming problem
$$x_{k} = argmin_{u\in\chi}\langle G_{v}^{k}, u\rangle$$ 
    3. Update z
$$z_{k+1} = (1-\alpha_{k})z_{k} + \alpha_{k} x_{k}$$ 

where: <br>
$z_{k}$ is our optimization parameter <br>
$\xi_{k}$ is a sample of our distribution <br>
$u_{k,j} \sim N(0, I_{d})$ <br>
$m_{k}$ is the number of gaussian vector to generate <br>
$\alpha_{k}$ is the momentum at time k <br>
$v$ is the gaussian smoothing parameter <br>

**Args:**

        Name            Type                Description
        x               (torch.tensor)      The variable of our optimization problem. Should be a 3D tensor (img)
        v               (float)             The gaussian smoothing
        n_gradient      (list)              Number of normal vector to generate at every step
        ak              (list)              Momentum  every step
        epsilon         (float)             The upper bound of norm
        L_type          (int)               Either -1 for L_infinity or x for Lx. Default is -1
        batch_size      (int)               Maximum parallelization during the gradient estimation. Default is -1 (=n_grad)
        C               (tuple)             The boundaires of the pixel. Default is (0, 1)
        max_steps       (int)               The maximum number of steps. Default is 100
        verbose         (int)               Display information or not. Default is 0
        additional_out  (bool)              Return also all the x. Default is False
        tqdm_disable    (bool)              Disable the tqdm bar. Default is False


     
     
**Suggested values:** <br>
$v = \sqrt{\frac{2B_{L_{\sigma}}}{N(d+3)^3}}$, 
$\alpha_{k} =\frac{1}{\sqrt{N}}$,
$m_{k} = 2B_{L_{\sigma}}(d + 5)N$,
$\forall k \geq 1$

where:<br>
- *N* is the number of steps <br>
- *d* is the dimension of *x* <br>
- $\sigma$ is the Strong Convexity coefficient
- $B \geq ||f(x)||, \forall x \in \chi$
- $B_{L_{\sigma}} ≥ max\bigg\{\sqrt{\frac{B^2 + \sigma^2}{L}}, 1\bigg\}$

**Empirical values:** <br>
In case of MNIST we can set:<br>
$N = 100$, $B_{L_{\sigma}} = 1$ and we have a image 28 * 28 ($d = 784$), so:<br>
- $v = 10e-6$
- $\alpha_{k} = 0.1$
- $m_{k} = 78900$

**N.B** <br>
In reality it seems that $\alpha_{k}$ could be set much higher (e.g. 0.3)  and $m_{k}$ could be set much lower (e.g. 600) and doesn't need to be dependent of the number of steps *N*.



In [1]:
import torch 
import numpy as np
from torch import nn
from tqdm import tqdm

class ClassicZSCG(object):
    """
    Args:
    Name            Type                Description
    model:          (nn.Module)         The model to use to get the output
    loss:           (nn.Module)         The loss to minimize
    device:
    """
    def __init__(self, model, loss, device=torch.device('cuda')):
        self.device = device
        self.loss = loss
        self.model = model.to(self.device)
        self.model.eval()

    """
    Perform an attack against the model given an input and the required run params
    """
    def run(self, x, v, n_gradient, ak , epsilon, L_type=-1, batch_size = -1, C = (0, 1), 
            max_steps=100, verbose=0, additional_out=False, tqdm_disabled=False):
        
        """
        Args:
        Name            Type                Description
        x               (torch.tensor)      The variable of our optimization problem. Should be a 3D tensor (img)
        v               (float)             The gaussian smoothing
        n_gradient      (list)              Number of normal vector to generate at every step
        ak              (list)              Momentum  every step
        epsilon         (float)             The upper bound of norm
        L_type          (int)               Either -1 for L_infinity or x for Lx. Default is -1
        batch_size      (int)               Maximum parallelization during the gradient estimation. Default is -1 (=n_grad)
        C               (tuple)             The boundaires of the pixel. Default is (0, 1)
        max_steps       (int)               The maximum number of steps. Default is 100
        verbose         (int)               Display information or not. Default is 0
        additional_out  (bool)              Return also all the x. Default is False
        tqdm_disable    (bool)              Disable the tqdm bar. Default is False
        """
        
        x = x.to(self.device)
        # 1. Init class attributes
        self.x_original = x.clone()
        self.dim = x.shape
        self.total_dim = torch.prod(torch.tensor(x.shape))
        self.epsilon = epsilon
        self.L_type = L_type
        self.C = C
        self.batch = batch_size
        

        # 2. Init list of results
        losses, outs = [], []
        x_list = []

        # 3. Main optimization cycle
        for ep in tqdm(range(max_steps), disable=tqdm_disabled):
            
            if verbose:
                print("---------------")
                print("Step number: {}".format(ep))
            
            # 3.1 Call the step
            x, gk = self.step(x, v, ak[ep], n_gradient[ep], verbose)
            x = x.reshape(self.dim[0], self.dim[1], self.dim[2]).detach()
            
            # 3.2 Compute loss
            out = self.model(x.view(1, self.dim[0], self.dim[1], self.dim[2]))
            loss = self.loss(out)
            
            # 3.3 Save results
            losses.append(loss.detach().cpu().item())
            outs.append(out.detach().cpu()[0, self.loss.neuron].item())
            if additional_out:
                x_list.append(x.cpu())
            
            # 3.4 Display current info
            if verbose:
                print("Loss:        {}".format(losses[-1]))
                print("Output:      {}".format(outs[-1]))
            
            # 3.5 Check Stopping criterions
            condition1 = (int(torch.argmax(out)) != self.loss.neuron) and (self.loss.maximise == 0)
            condition2 = (int(torch.argmax(out)) == self.loss.neuron) and (self.loss.maximise == 1)
            if condition1 or condition2:
                break

        if additional_out:
            return x, losses, outs, x_list
        return  x, losses, outs

    
    """
    Do an optimization step
    """
    def step(self, x, v, ak, mk, verbose=0):
        """
        Args:
        Name            Type                Description
        x:              (torch.tensor)      The variable of our optimization problem. Should be a 3D tensor (img)
        v:              (float)             The gaussian smoothing
        ak:             (float)             The weight avarage in the updating phase. (1 means take only the new x, 0 means no update)
        mk:             (int)               The number of Gaussian Random Vector to generate
        verbose:        (bool)              Display information or not. Default is 0
        """

        # Compute the approximated gradient
        g = self.compute_Gk(x, v, mk, verbose)
        
        # Call the inexact conditional gradient
        x_g = self.compute_CG(g, verbose).reshape(x.shape[0], x.shape[1], x.shape[2])
        x_new = (1-ak)*x + ak*x_g

        if verbose > 1:
            print("\nINSIDE STEP")
            print("Gradient has shape: {}".format(g.shape))
            print("Gradient is:\n{}".format(g))
            print("x_new has shape: {}".format(x_new.shape))
            print("x_new is:\n{}".format(x_new))

        return x_new.detach(), g.detach()

    
    """
    Prepare parallelization for the gradient estimation
    """
    def get_parallel(self, bs):
        """
        Args:
        Name            Type                Description
        bs              (int)               The maximum bacth size
        """
        uk     = torch.empty(bs, self.total_dim).normal_(mean=0, std=1).to(self.device) # Dim (bs, channel*width*height)
        img_u  = uk.reshape(bs, self.dim[0], self.dim[1], self.dim[2])                  # Dim (bs, channel, width, height)
        img_x  = x.expand(bs, self.dim[0], self.dim[1], self.dim[2])                    # Dim (bs, channel, width, height)
        m_x    = (img_x + v*img_u)
        
        return m_x, uk

    """
    Compute the Gv(x(k-1), chi(k-1), u(k)) in order to compute an approximation of the gradient of f(x(k-1), chi(k-1))
    """
    def compute_Gk(self, x, v, mk, verbose=0):
        """
        Args:
        Name            Type                Description
        x:              (torch.tensor)      The variable of our optimization problem. Should be a 3D tensor (img)
        v:              (float)             The gaussian smoothing
        mk:             (int)               The number of Gaussian Random Vector to generate
        verbose:        (bool)              Display information or not. Default is 0
        """

        # 1. Get objective functions

        if self.batch == -1:
            
            # 1.a Compute standard Loss
            standard_loss = self.loss(self.model(x.view(1, *list(self.dim)))) 
            
                                                               
            # 1.b Compute gaussian loss
            uk, m_x = self.get_parallel(mk)
            gaussian_loss = self.loss(self.model(m_x))
            
            if verbose > 1:
                print('\nINSIDE GRADIENT')
                print('The Gaussian vector uk has shape:{}'.format(uk.shape))
                print('The input x has shape:\t\t{}'.format(x.shape))
                print('The input x + vu has shape:\t{}'.format(m_x.shape))


            # 1.c Compute Gv(x(k-1), chi(k-1), u(k))
            fv = ((gaussian_loss - standard_loss.expand(uk.shape[0]))/v).view(-1, 1)        # Dim (mk, 1)
            G = fv * uk                                                                     # Dim (mk, channel*width*height)

            return torch.mean(G, axis=0).detach()

        else:
            
            # 1.a Compute standard loss
            standard_loss = self.loss(self.model(x.view(1, *list(self.dim))))                   # Dim (1)
            G_tot = torch.zeros(mk//self.batch, self.total_dim).to(self.device)                 # Dim (n_batches, hannel*width*height)

            #1.b Compute Gradient
            for n in range(mk//self.batch):
                from_, to_ = n*self.batch, (n+1)*self.batch

                # 1.b Create batch x(k-1) + v*u(k-1)
                uk, mk = self.get_parallel(self.batch)

                # 1.c Compute
                tmp_gaussian_loss = self.loss(self.model(m_x)).detach()                                 # Dim(bs)
                
                if verbose > 1:
                    print('\nINSIDE GRADIENT')
                    print('The Gaussian vector uk has shape:{}'.format(uk.shape))
                    print('The input x has shape:\t\t{}'.format(x.shape))
                    print('The input x + vu has shape:\t{}'.format(m_x.shape))
                
                # 1.d Compute Gradient
                fv = ((tmp_gaussian_loss - standard_loss.expand(uk.shape[0]))/v).view(-1, 1)            # Dim (bs, 1)
                G = fv * uk                                                                             # Dim (bs, channel*width*height)

                if verbose > 1:
                    print('Gaussian cycle loss has shape:\t{}'.format(tmp_gaussian_loss.shape))
                    print('Function approx has shape:\t{}'.format(fv.shape))
                    print('Gradient has shape:\t\t{}'.format(G.shape))

                G_tot[n] = torch.mean(G, axis=0).detach()

        return torch.mean(G_tot, axis=0).detach()


    def compute_CG(self, g, verbose):
        """
        Args:
        Name         Type                Description
        g:           (torch.tensor)      The approximated gradient. Should be a 1D tensor
        """

        # Infinity norm
        if self.L_type == -1:
            x_new = self.x_original.view(-1) - self.epsilon*torch.sign(g)
            
        # L1 norm
        elif self.L_type == 1:
            raise NotImplementedError
            
        elif self.L_type == 2:
            x_new = self.x_original.view(-1) - (self.epsilon*g)/torch.norm(g, 2)
            
        # Generic Lp norm (1 < p < +inf)
        else:
            p = self.L_type
            gp = torch.abs(g)**(1/p-1)
            h = torch.sign(g) * (gp) / torch.norm(gp, p)
            x_new = self.x_original.view(-1) - self.epsilon*h

        # Enforce pixel boundaries
        x_new[x_new < self.C[0]] = self.C[0]
        x_new[x_new > self.C[1]] = self.C[1]

        if verbose > 1:
            print('\nINSIDE CG')
            print('Epsilon * Sign(g) is {}'.format(self.epsilon*torch.sign(g)))
            print('Unchecked new x is: {}'.format(self.x_original.view(-1) - self.epsilon*torch.sign(g)))
            print('The CG gradient is:\n{}'.format(g))
            print('The new x is:\n {}'.format(x_new))

        return x_new.detach()

In [ ]:
class InexactZSCG(object):
    """
    Args:
    Name            Type                Description
    model:          (nn.Module)         The model to use to get the output
    loss:           (nn.Module)         The loss to minimize
    device:
    """
    def __init__(self, model, loss, device=torch.device('cuda')):
        self.device = device
        self.loss = loss
        self.model = model.to(self.device)
        self.model.eval()

    
    """
    Perform an attack against the model given an input and the required run params
    """
    def run(self, x, v, n_gradient, gamma_k, mu_k, epsilon, L_type = -1, batch_size = -1, C = (0, 1), 
            max_steps=100, verbose=0, additional_out=False, tqdm_disabled=False, max_t=1000):
        """
        Args:
        Name            Type                Description
        x               (torch.tensor)      The variable of our optimization problem. Should be a 3D tensor (img)
        v               (float)             The gaussian smoothing
        n_gradient      (list)              Number of normal vector to generate at every step
        gamma_k         (list)              Momentum at every step inside ICG
        mu_k            (list)              Stoppinc criterion at every step k inside ICG
        max_t           (int)               The maximum number of iteration inside of ICG.
        epsilon         (float)             The upper bound of norm
        L_type          (int)               Either -1 for L_infinity or x for Lx. Default is -1
        batch_size      (int)               Maximum parallelization during the gradient estimation. Default is -1 (=n_grad)
        C               (tuple)             The boundaires of the pixel. Default is (0, 1)
        max_steps       (int)               The maximum number of steps. Default is 100
        verbose         (int)               Display information or not. Default is 0
        additional_out  (bool)              Return also all the x. Default is False
        tqdm_disable    (bool)              Disable the tqdm bar. Default is False    
        """
        
        x = x.to(self.device)
        
        # 1. Init class attributes
        self.x_original = x.clone()
        self.dim = x.shape
        self.total_dim = torch.prod(torch.tensor(x.shape))
        self.epsilon = epsilon
        self.L_type = L_type
        self.C = C
        self.batch = batch_size
        self.max_t = max_t

        # 2. Init list of results
        losses, outs = [], [] 
        x_list = []

        # 3. Main optimization cycle
        for ep in tqdm(range(max_steps), disable=tqdm_disabled):
            
            if verbose:
                print("---------------")
                print("Step number: {}".format(ep))
            
            # 3.1 Call the step
            x, gk = self.step(x, v, gamma_k[ep], mu_k[ep], n_gradient[ep], verbose)
            x = x.reshape(self.dim[0], self.dim[1], self.dim[2]).detach()
            
            # 3.2 Compute loss
            out = self.model(x.view(1, self.dim[0], self.dim[1], self.dim[2]))
            loss = self.loss(out).view(-1, 1)
            
            # 3.3 Save results
            losses.append(loss.detach().cpu().item())
            outs.append(out.detach().cpu()[0, self.loss.neuron].item())
            if additional_out:
                x_list.append(x.cpu())
            
            # 3.4 Display current info
            if verbose:
                print("Loss:        {}".format(losses[-1]))
                print("Output:      {}".format(outs[-1]))
                
            # 3.5 Check Stopping criterion
            condition1 = (int(torch.argmax(out)) != self.loss.neuron) and (self.loss.maximise == 0)
            condition2 = (int(torch.argmax(out)) == self.loss.neuron) and (self.loss.maximise == 1)
            if condition1 or condition2:
                break

        if additional_out:
            return x, losses, outs, input_list
        return  x, losses, outs

    """
    Do an optimization step
    """
    def step(self, x, v, gamma, mu, mk, verbose=0):
        """
        Args:
        Name            Type                Description
        x:              (torch.tensor)      The variable of our optimization problem. Should be a 3D tensor (img)
        v:              (float)             The gaussian smoothing
        gamma:          (float)             The update parameters of g
        mu:             (float)             The stopping criterion
        mk:             (int)               The number of Gaussian Random Vector to generate
        verbose:        (bool)              Display information or not. Default is 0
        """
        # Compute the approximated gradient
        g = self.compute_Gk(x, v, mk, verbose)
        # Call the inexact conditional gradient
        x_new = self.compute_ICG(x, g, gamma, mu, verbose).reshape(x.shape[0], x.shape[1], x.shape[2])
    
        if verbose > 1:
            print("\nINSIDE STEP")
            print("Gradient has shape: {}".format(g.shape))
            print("Gradient is:\n{}".format(g))
            print("x_new has shape: {}".format(x_new.shape))
            print("x_new is:\n{}".format(x_new))

        return x_new.detach(), g.detach()
    
    
    """
    Prepare parallelization for the gradient estimation
    """
    def get_parallel(self, x, bs, v):
        """
        Args:
        Name            Type                Description
        x               (torch.tensor)      The current variable
        bs              (int)               The maximum bacth size
        v               (int)               The Gaussian smoothing
        """
        uk     = torch.empty(bs, self.total_dim).normal_(mean=0, std=1).to(self.device) # Dim (bs, channel*width*height)
        img_u  = uk.reshape(bs, self.dim[0], self.dim[1], self.dim[2])                  # Dim (bs, channel, width, height)
        img_x  = x.expand(bs, self.dim[0], self.dim[1], self.dim[2])                    # Dim (bs, channel, width, height)
        m_x    = (img_x + v*img_u)

        return m_x, uk

    """
    Compute the Gv(x(k-1), chi(k-1), u(k)) in order to compute an approximation of the gradient of f(x(k-1), chi(k-1))
    """
    def compute_Gk(self, x, v, mk, verbose=0):
        """
        Args:
        Name            Type                Description
        x:              (torch.tensor)      The variable of our optimization problem. Should be a 3D tensor (img)
        v:              (float)             The gaussian smoothing
        mk:             (int)               The number of Gaussian Random Vector to generate
        verbose:        (bool)              Display information or not. Default is 0
        """

        # 1. Get objective functions
        # CASE BATCH_SIZE == N_GRADIENT
        if self.batch == -1:

            # 1.a Compute standard Loss
            standard_loss = self.loss(self.model(x.view(1, *list(self.dim))))

            # 1.b Compute gaussian loss
            m_x, uk = self.get_parallel(x, mk, v)
            gaussian_loss = self.loss(self.model(m_x))

            if verbose > 1:
                print('\nINSIDE GRADIENT')
                print('The Gaussian vector uk has shape:{}'.format(uk.shape))
                print('The input x has shape:\t\t{}'.format(x.shape))
                print('The input x + vu has shape:\t{}'.format(m_x.shape))

            # 1.c Compute Gv(x(k-1), chi(k-1), u(k))
            fv = ((gaussian_loss - standard_loss.expand(uk.shape[0]))/v).view(-1, 1)        # Dim (mk, 1)
            G = fv * uk                                                                     # Dim (mk, channel*width*height)

            return torch.mean(G, axis=0).detach()

        # CASE BATCH_SIZE < N_GRADIENT
        else:

            # 1.a Compute standard loss
            standard_loss = self.loss(self.model(x.view(1, *list(self.dim))))                   # Dim (1)
            G_tot = torch.zeros(mk//self.batch, self.total_dim).to(self.device)                 # Dim (n_batches, hannel*width*height)

            #1.b Compute Gradient
            for n in range(mk//self.batch):
                from_, to_ = n*self.batch, (n+1)*self.batch

                # 1.b Create batch x(k-1) + v*u(k-1)
                m_x, uk = self.get_parallel(x, self.batch, v)

                # 1.c Compute
                tmp_gaussian_loss = self.loss(self.model(m_x)).detach()                                 # Dim(bs)

                if verbose > 1:
                    print('\nINSIDE GRADIENT')
                    print('The Gaussian vector uk has shape:{}'.format(uk.shape))
                    print('The input x has shape:\t\t{}'.format(x.shape))
                    print('The input x + vu has shape:\t{}'.format(m_x.shape))

                # 1.d Compute Gradient
                fv = ((tmp_gaussian_loss - standard_loss.expand(uk.shape[0]))/v).view(-1, 1)            # Dim (bs, 1)
                G = fv * uk                                                                             # Dim (bs, channel*width*height)

                if verbose > 1:
                    print('Gaussian cycle loss has shape:\t{}'.format(tmp_gaussian_loss.shape))
                    print('Function approx has shape:\t{}'.format(fv.shape))
                    print('Gradient has shape:\t\t{}'.format(G.shape))

                G_tot[n] = torch.mean(G, axis=0).detach()

        return torch.mean(G_tot, axis=0).detach()
    
    

    """
    Compute the Inexact Condtion Gradient (Algorithm 3 of source article)
    """
    def compute_ICG(self, x, g, gamma, mu, verbose):
        """
        Args:
        Name            Type                Description
        x:              (torch.tensor)      The variable of our optimization problem. Should be a 3D tensor (img)
        g:              (torch.tensor)      The approximated gradient. Should be a 1D tensor
        gamma:          (float)             The update parameters of g
        mu:             (float)             The stopping criterion
        """
        # 1. Init variables
        y_old = x.view(-1).clone() # dim = (n_channel * width * height)
        u = torch.rand(self.total_dim).to(self.device)*(self.max.view(-1) - self.min.view(-1)) + self.min.view(-1)
        t = 1
        k = 0

        # 2. Main cycle
        while(k==0):
            
            # 2.1 Compute gradient
            grad = g + gamma*(y_old - x.view(-1))
            
            # 2.2 Perform LMO
            # Infinity norm
            if self.L_type == -1:
                x_new = self.x_original.view(-1) - self.epsilon*torch.sign(g)
            # L1 norm
            elif self.L_type == 1:
                raise NotImplementedError
            elif self.L_type == 2:
                x_new = self.x_original.view(-1) - (self.epsilon*g)/torch.norm(g, 2)
            # Generic Lp norm (1 < p < +inf)
            else:
                p = self.L_type
                gp = torch.abs(g)**(1/p-1)
                h = torch.sign(g) * (gp) / torch.norm(gp, p)
                x_new = self.x_original.view(-1) - self.epsilon*h
                
            # 2.3 Compute new function value
            h = torch.dot(grad, y_new - y_old)

            if verbose > 1:
                print('\nINSIDE ICG')
                print('Time t = {}'.format(t))
                print('The ICG gradient is:\n{}'.format(grad))
                print('The new y is:\n {}'.format(y_new))
                print('The function h(y_new) is {}'.format(h))
                print('Mu is: {}'.format(mu))
                
            # 2.4 Check conditions
            if h >= -mu or t > self.max_t:
                k = 1
            else:
                y_old = (t-1)/(t+1)*y_old + 2/(t+1)*y_new
                t += 1

        return self.project_boundaries(y_old.detach())


    """
    Check the boundaries of our constraint optimization problem
    """
    def project_boundaries(self, x):
        x[x > self.C[1]] = self.C[1]
        x[x < self.C[0]] = self.C[0]
        return x